## Домашнее задание по неделе 4

Как было рассказано на лекции, стохастический градиентый спуск сходится быстрее, чем полный, хотя и менее стабильно. В этом задании вам предлагается реализовать стохастический градиентный спуск и сравнить его с точным вычислением весов линейной модели по скорости работы и значению метрики качества.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

np.random.seed(0)

warnings.filterwarnings('ignore')
%matplotlib inline

### Задание 0

Реализуйте класс ```LinearRegressionSGD``` c обучением и и применением линейной регрессии, построенной с помощью стохастического градиентного спуска, с заданным интерфейсом.

In [3]:
from sklearn.base import BaseEstimator

class LinearRegressionSGD_my(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=100, w0=None, alpha=1e-4, batch_size=1):
        """
        epsilon: разница для нормы изменения весов
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        """
        if batch_size <= 0:
          raise Exception('batch_size can only be greater than zero')
        
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.batch_size = batch_size
        self.w_history = []

    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        l, d = X.shape
        if self.w0 is None:
          self.w0 = np.zeros(d)

        self.w = self.w0

        for step in range(self.max_steps):
          self.w_history.append(self.w)
          indices = np.random.choice(l, self.batch_size, replace=False)
          X_batch = X.iloc[indices]
          y_batch = y.iloc[indices]
          w_new = self.w - self.alpha * self.calc_gradient(X_batch, y_batch)
          if (np.linalg.norm(w_new - self.w) < self.epsilon):
            break

          self.w = w_new
        return self

    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """
        if self.w is None:
            raise Exception('Not trained yet')

        return np.dot(X, self.w)

    def calc_gradient(self, X_i, y_i):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: np.array (d)
        """
        
        l, d = X_i.shape
        return (2/l) * np.dot(X_i.T,(np.dot(X_i, self.w) - y_i))

Проверять работу мы будем на имеющемся в sklearn наборе данных.
Возьмем стандартный [датасет](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) со стоимостью жилья в различных районах Калифорнии в 1990 году.  Датасет содержит информацию о средних ценах на жилье в районе и какие-то параметры района: средний возраст домов, среднее число комнат, население

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing(as_frame=True)
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [41]:
X_train.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
17853,5.3994,23.0,5.019157,1.022989,910.0,3.486590,37.44,-121.88
15963,3.9567,52.0,5.173664,1.127863,1848.0,3.526718,37.71,-122.44
20106,3.0500,17.0,5.383764,1.095941,753.0,2.778598,37.94,-120.29
15525,2.2500,16.0,4.331113,1.109420,2737.0,2.604186,33.14,-117.05
5234,2.0187,39.0,4.876068,1.102564,1313.0,5.611111,33.94,-118.23


### Задание 1

Метрикой качества в нашей задаче будет MAPE - Mean Absolute Percentage Error. Реализуйте её с заданным интефейсом и вычислите
```MAPE(y_test, y_0)```, где ```y_0 = (mean(y_test), mean(y_test), ...)```

In [42]:
def MAPE(y_true, y_pred):
    """
        y_true: np.array (l)
        y_pred: np.array (l)
        ---
        output: float [0, +inf)
    """
    l = len(y_true)
    l_pred = len(y_pred)
    if l != l_pred:
        raise Exception(f'Different length: length y_true = {l};length y_pred = {l_pred}')
    mape = 0
    
    epsilon = 1e-10
    absolute_percentage_error = np.abs((y_true - y_pred) / (y_true + epsilon))
    mape = np.mean(absolute_percentage_error)
    
    return mape

In [43]:
y_0 = np.repeat([y_test.mean()], len(y_test))
MAPE(y_test, y_0)

0.6221208474158171

### Задание 2

Обучите ```LinearRegressionSGD``` с базовыми параметрами на тренировочном наборе данных (```X_train```, ```y_train```), сделайте предсказание на тестовых данных ```X_test```, записав результат в переменную ```y_pred_sgd```  и вычислите ошибку MAPE.

In [45]:
from sklearn.linear_model import SGDRegressor

In [53]:
sgd = SGDRegressor()
sgd.fit(X_train, y_train)
y_pred_sgd = sgd.predict(X_test)
print(MAPE(y_test, y_pred_sgd) * 100)
# print(f"test = {y_test}")
# print(f"pred = {y_pred_sgd}")

8452939975000225.0


### Задание 3

Вычислите веса по точной формуле, используя ```X_train``` и ```y_train```; предскажите с их помощью целевую переменную на ```X_test```, записав результат в переменную ```y_pred_lr``` и вычислите ошибку MAPE.

In [9]:
class LinearRegressionVectorized(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=1000, w0=None, alpha=1e-2):
        """
        epsilon: разница для нормы изменения весов
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.w_history = []

    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        l, d = X.shape

        if self.w0 is None:
          self.w0 = np.zeros(d)

        self.w = self.w0

        for step in range(self.max_steps):
          self.w_history.append(self.w)

          w_new = self.w - self.alpha * self.calc_gradient(X, y)

          if (np.linalg.norm(w_new - self.w) < self.epsilon):
            break

          self.w = w_new

        return self

    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """

        if self.w is None:
            raise Exception('Not trained yet')

        return np.dot(X, self.w)


    def calc_gradient(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: np.array (d)
        """
        l, d = X.shape

        return (2/l) * np.dot(X.T,(np.dot(X, self.w) - y))


In [54]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print(lr.coef_)
round(MAPE(y_test, y_pred_lr) * 100, 1)


[ 4.40924382e-01  9.67539152e-03 -1.14289456e-01  7.58983172e-01
  1.30806961e-06 -4.18120482e-03 -4.32535255e-01 -4.45125707e-01]


31.8

In [296]:
lrm = LinearRegressionVectorized()
lrm.fit(X_train.to_numpy(), y_train.to_numpy())

LinearRegressionVectorized(w0=array([0., 0., 0., 0., 0., 0., 0., 0.]))

## Бонусное задание по неделе 4

До этого вы релизовывали модели, в которых не было штрафа за величину весов ```w```. Как было рассказано ранее в лекциях, это может привести к неустойчивости модели и переобучению. Чтобы избежать этих эффектов, предлагается добавить к оптимизируемому функционалу L2-норму весов; таким образом, будем решать задачу гребневой регрессии, Ridge:

$$ \frac{1}{l}(Xw-y)^T(Xw-y) +\gamma||w||_2 \rightarrow \min_{w}. $$

### Задание 11
Реализуйте обучение такой модели в матричном виде (смотрите дополнительные материалы к этой неделе) с помощью стохастического градиентного спуска. Класс должен совпадать по набору реализованных функций с ```LinearRegressionVectorized```, разница будет лишь в параметре $\gamma$, отвечающем за степень регуляризации.

In [ ]:
class RidgeSGD(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=1000, w0=None, alpha=1e-2, gamma=0):
        """
        epsilon: разница для нормы изменения весов
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        gamma: коэффициент регуляризации
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.gamma = gamma
        self.w = None
        self.w_history = []

    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        return self

    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """
        pass


    def calc_gradient(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: np.array (d)
        """
        pass

Так же, как и в основном задании, обучите модель с базовыми параметрами на тренировочных данных и сделайте прогноз y_pred_ridge. Выведите значение MAPE(y_test, y_pred_ridge).